In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.layers import Dropout

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cap-4611-2021-fall-assignment-4/sample_submission.csv
/kaggle/input/cap-4611-2021-fall-assignment-4/eval.csv
/kaggle/input/cap-4611-2021-fall-assignment-4/train.csv


## Read CSV

In [2]:
train_df = pd.read_csv("../input/cap-4611-2021-fall-assignment-4/train.csv", low_memory = False)
test_df = pd.read_csv("../input/cap-4611-2021-fall-assignment-4/eval.csv", low_memory = False)

## Exploratory Data Analysis

In [3]:
train_df.head()

,id,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,10000,4,0,0,0,0,0,0,1,1,...,111,81,0,0,0,3,28,0,0,0
1,10001,5,0,0,0,0,0,0,0,0,...,0,0,0,4,0,45,99,0,0,0
2,10002,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,6,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
4,10004,8,0,0,0,0,0,0,0,0,...,133,125,109,90,42,0,0,0,0,0


In [4]:
test_df.head()

,id,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,2,...,0,0,0,6,217,220,141,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,75,74,109,55,0,3,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
print(train_df.shape)
print(test_df.shape)

(60000, 786)
(10000, 785)


In [6]:
print(train_df.ndim)
print(test_df.ndim)

2
2


In [7]:
print(train_df.shape)
print(test_df.shape)

(60000, 786)
(10000, 785)


In [8]:
print(train_df.isna().sum().value_counts())
print(test_df.isna().sum().value_counts())

0    786
dtype: int64
0    785
dtype: int64


## Train Test Split

In [9]:
y = train_df['label']
X = train_df.iloc[:,2:786].to_numpy().reshape(len(train_df), 28, 28, 1)

test_modified = test_df.iloc[:,1:785].to_numpy().reshape(len(test_df), 28, 28, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [10]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(54000, 28, 28, 1)
(54000,)
(6000, 28, 28, 1)
(6000,)


## Model

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", input_shape = [28, 28, 1]))
model.add(keras.layers.Conv2D(128, 3, activation = "relu", padding = "same"))
model.add(keras.layers.Conv2D(256, 3, activation = "relu", padding = "same"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(150, activation = "relu"))
model.add(keras.layers.Dense(100, activation = "relu"))
model.add(keras.layers.Dense(50, activation = "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [13]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 150)               30105750  
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5

In [15]:
print(X_train.ndim)
print(y_train.ndim)
print(X_test.ndim)
print(y_test.ndim)

4
1
4
1


In [16]:
history = model.fit(X_train, y_train, batch_size = 100, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10


2021-11-22 02:02:39.768483: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


540/540 [==============================] - 907s 2s/step - loss: 1.1431 - accuracy: 0.7827 - val_loss: 0.3277 - val_accuracy: 0.8833
Epoch 2/10
540/540 [==============================] - 897s 2s/step - loss: 0.3815 - accuracy: 0.8720 - val_loss: 0.3119 - val_accuracy: 0.8890
Epoch 3/10
540/540 [==============================] - 899s 2s/step - loss: 0.3156 - accuracy: 0.8916 - val_loss: 0.2895 - val_accuracy: 0.8960
Epoch 4/10
540/540 [==============================] - 899s 2s/step - loss: 0.2669 - accuracy: 0.9053 - val_loss: 0.2938 - val_accuracy: 0.9047
Epoch 5/10
540/540 [==============================] - 903s 2s/step - loss: 0.2413 - accuracy: 0.9142 - val_loss: 0.3085 - val_accuracy: 0.9008
Epoch 6/10
540/540 [==============================] - 900s 2s/step - loss: 0.2077 - accuracy: 0.9255 - val_loss: 0.3202 - val_accuracy: 0.9030
Epoch 7/10
540/540 [==============================] - 899s 2s/step - loss: 0.1874 - accuracy: 0.9331 - val_loss: 0.3710 - val_accuracy: 0.8997
Epoch 8/10

## Output

In [17]:
test_modified = test_df.iloc[:,1:785].to_numpy().reshape(len(test_df), 28, 28, 1)
predictions = model.predict(test_modified).argmax(axis = 1)

print(predictions.shape)

(10000,)


In [18]:
output = pd.DataFrame({'id': test_df['id'], 'label': predictions})
output.to_csv('submission.csv', index = False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
